<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/budget_forecast_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import logging
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

In [2]:
# Set Prophet loggers to WARNING level
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

In [3]:
# cleaning budget unit 7001
def clean_7001(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df['Proj/Ativ'] == 2870, 'Proj/Ativ'] = 4396
  df.loc[df['Proj/Ativ'].isin([2872, 1507]), ['Proj/Ativ', 'Elemento']] = 4471, 339040
  df.loc[df['Proj/Ativ'].isin([2873, 2532]), 'Proj/Ativ'] = 4413
  df.loc[df['Proj/Ativ'].isin([1505, 1503, 1373, 1506]), 'Proj/Ativ'] = 2529
  df.loc[(df['Proj/Ativ'] == 2681) & (df['Elemento'] == 319192), 'Elemento'] = 319113
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'] == 449092), 'Elemento'] = 449051
  df.loc[(df['Proj/Ativ'] == 9071) & (df['Elemento'] == 319091), 'Elemento'] = 339091
  df.loc[(df['Proj/Ativ'] == 9071) & (df['Elemento'].isin([339092, 339147])), 'Elemento'] = 339047
  df.loc[~((df['Proj/Ativ'] == 9071) & (df['Vinc. Orçam.'] == 1)), 'Vinc. Orçam.'] = 6069
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'].isin([319011, 319016, 319092, 319094, 339036, 339046, 339049])), 'Proj/Ativ'] = 4396
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'] == 319013), 'Proj/Ativ'] = 2680
  return df


# changing the elements 339001, 339003, 339091, 339092, 332001
def change_elements(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df['Elemento'] == 339001, 'Elemento'] = 319001
  df.loc[df['Elemento'] == 339003, 'Elemento'] = 319003
  df.loc[df['Elemento'] == 339091, 'Elemento'] = 319091
  df.loc[(df['Elemento'] == 339092) & (~df['Proj/Ativ'].isin([9075, 9077])), 'Elemento'] = 319092
  df.loc[df['Elemento'] == 332001, 'Elemento'] = 339086
  return df


# filling empty cells after the first filled cell in a column with 0
def fill_zero(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns[1:]:
        first_valid = df[col].first_valid_index()
        if first_valid is not None:
            df.loc[first_valid:, col] = df.loc[first_valid:, col].fillna(0)
    return df

# creating moving average dataframes
def moving_averages(df: pd.DataFrame, window: int) -> pd.DataFrame:
    df_ma = df.copy()
    df_ma.loc[:, df_ma.columns.difference(['T', 'Comp.pagto.'])] = df_ma.loc[:, df_ma.columns.difference(['T', 'Comp.pagto.'])].rolling(window).mean()
    df_ma.dropna(axis=1, how='all', inplace=True)
    return df_ma

# building prophet model
def build_prophet_model() -> Prophet:
  model = Prophet(
      # growth='logistic',
      yearly_seasonality=False,
      weekly_seasonality=False,
      daily_seasonality=False
  )
  model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
  model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
  return model

# normalizes and calculates the score
def normalizes_calculates_score(df: pd.DataFrame) -> pd.DataFrame:
  scaler = MinMaxScaler()
  df['r2_norm'] = scaler.fit_transform(df[['R²']])
  df['rmse_norm'] = 1 - scaler.fit_transform(df[['RMSE']])
  df['mae_norm'] = 1 - scaler.fit_transform(df[['MAE']])

  df['score'] = (
      0.5 * df['r2_norm'] +
      0.25 * df['rmse_norm'] +
      0.25 * df['mae_norm']
  )
  return df

mass_segregation_cols = {
    '7002-2736-319003-6049', '7002-2738-319003-6049', '7002-2740-319003-6049', '7002-2742-319003-6049',
    '7002-2744-319003-6049', '7002-2747-319003-6049', '7002-2752-319003-6049', '7002-2754-319003-6049',
    '7002-2756-319003-6049', '7003-2760-319003-6050', '7003-2762-319003-6050', '7003-2764-319003-6050',
    '7003-2766-319003-6050', '7003-2768-319003-6050', '7003-2771-319003-6050', '7003-2776-319003-6050',
    '7003-2778-319003-6050', '7003-2780-319003-6050'
}

# def time_based_train_test_split(df: pd.DataFrame, min_len: int = 2) -> tuple[pd.DataFrame, pd.DataFrame | None]:
#     """
#     Perform a time-based train/test split.

#     Args:
#         df: DataFrame with time-ordered data.
#         min_len: Minimum length required to return a split.

#     Returns:
#         (train_df, test_df): Tuple of train and test DataFrames. Test can be None.
#     """
#     if len(df) >= 60:
#         train_df = df[:-12]
#         test_df = df[-12:]
#     elif len(df) >= 12:
#         split_idx = int(len(df) * 0.8)
#         train_df = df.iloc[:split_idx]
#         test_df = df.iloc[split_idx:]
#     elif len(df) >= min_len:
#         train_df = df
#         test_df = None
#     else:
#         return None, None

#     return train_df.reset_index(drop=True), test_df.reset_index(drop=True) if test_df is not None else None

# def str_to_float(value: str) -> float:
#   value = str(value).replace('.', '').replace(',', '.')
#   value = re.sub(r'[A-Za-z]|\s', '', value)
#   return float(0) if value == '' else float(value)

# def agrupamento_format(value: str) -> str:
#   if re.search(r'^0{2}', value[:2]):
#     value = '   ' + value
#   elif re.search(r'^[1-9]\d', value[:2]):
#     value = '      ' + value
#   return value

# dict_month = {'Janeiro': '01', 'Fevereiro': '02', 'Março': '03', 'Abril': '04',
#               'Maio': '05', 'Junho': '06', 'Julho': '07', 'Agosto': '08',
#               'Setembro': '09', 'Outubro': '10', 'Novembro': '11', 'Dezembro': '12'}

In [4]:
# getting expense data
df_exp = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_exp.csv', sep=';', parse_dates=['Comp.pagto.'])

# getting budget settlement data from 2021
df_2021 = pd.read_excel(r'/content/drive/MyDrive/Previsoes_orcamento/despesas/2021_liquidacao/Liquidações Consolidado.xls', parse_dates=['Compet.Liq.'])
df_2021['Compet.Liq.'] = pd.to_datetime(df_2021['Compet.Liq.'], dayfirst=True)
df_2021 = df_2021.loc[
    (df_2021['Compet.Estorno'] == '21/12/2021') & (df_2021['Unid.Orçam.'] == 7002),
    ['Compet.Liq.', 'Unid.Orçam.', 'Proj/Ativ', 'Rubrica', 'Vinc.Orçam.', 'Val. Liquidado']
    ].copy().rename(columns={
        'Compet.Liq.': 'Comp.pagto.',
        'Unid.Orçam.': 'Unid. Orçam.',
        'Vinc.Orçam.': 'Vinc. Orçam.',
        'Val. Liquidado': 'Result. pago'
    })
df_2021['Elemento'] = df_2021['Rubrica'].astype(str).str[:6].astype(int)

# deleting specific 2021 data to replace it with budget settlement data
df_exp = df_exp[~((df_exp['Comp.pagto.'] == '2021-12-29') & (df_exp['Unid. Orçam.'] == 7002))]
df_exp = pd.concat([df_exp, df_2021], ignore_index=True)

# correcting data
df_exp.loc[
    (df_exp['Proj/Ativ'] == 2529) & (df_exp['Rubrica'] == 339036040000),
    ['Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
] = 7003, 9075, 339039, 339039030000, 6050
df_exp.loc[
    (df_exp['Proj/Ativ'] == 9042) & (df_exp['Comp.pagto.'].dt.year > 2010),
    ['Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica']
] = 7002, 9076, 339086, 339086010000

# dropping 'Rubrica" column
df_exp.drop('Rubrica', axis=1, inplace=True)

In [5]:
# Selecting and clearing data from 7001
df_7001 = clean_7001(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7001) &
    (df_exp['Vinc. Orçam.'].isin([400, 1, 6050, 6069])) &
    (df_exp['Comp.pagto.'].dt.year > 2011) # period prior to GPREV removed
].copy())

# Selecting and clearing data from 7002
df_7002 = change_elements(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7002) &
    (~df_exp['Proj/Ativ'].isin([2737, 2739, 2741, 2743, 2745, 2746, 2748, 2750, 2753, 2755, 2757, 2759]))
].copy())
df_7002['Vinc. Orçam.'] = 6049

# Selecting and clearing data from 7003
df_7003 = change_elements(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7003) &
    (~df_exp['Proj/Ativ'].isin([2761, 2763, 2765, 2767, 2769, 2770, 2772, 2774, 2777, 2779, 2781, 2783]))
].copy())
df_7003['Vinc. Orçam.'] = 6050

In [6]:
# getting revenue data
df_rev = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_rev.csv', sep=';', parse_dates=['Data'])

# clearing revenue data
df_rev.drop(columns=['origem', 'tipo'], inplace=True)
df_rev = df_rev[(df_rev['vinculo'].isin([6050, 6069, 6049, 400])) & (df_rev['Data'].dt.year > 2017)]
df_rev.loc[df_rev['vinculo'] == 400, 'vinculo'] = 6049
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Compensações Financ entre o Regime Geral e os RPPS'),
    'nome_rubrica'] = 'Compensações Financ entre o Regime Geral e os RPPS-Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Compensações Financ entre o Regime Geral e os RPPS'),
    'nome_rubrica'] = 'Comp. Financ. entre o Regime Geral e os RPPS - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Cedido - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Cedido - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo - Cedido - Multas e Juros - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Dív.At.- Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Ativo Cedido-Dív.Ativa-Multas e Juros - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Dívida Ativa'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo - Cedido - Dívida Ativa - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo-Cedido-Multas e Juros - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.Patronal - Serv. Afastados - Plano em Repartição'),
    'nome_rubrica'] = 'Contr.Patronal - Servidores Afastados - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Afastado'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Afastado - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Afastado'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Afastado - Plano em Capitalização'

df_rev = df_rev[~((df_rev['nome_rubrica'].str.lower().str.contains('patr')) & (df_rev['nome_rubrica'].str.lower().str.contains('inativo')))]
df_rev = df_rev[~df_rev['nome_rubrica'].str.contains('637|750|805|supl', case=False, regex=True)]


# df_pred_rev = df_rev.copy().rename(columns={'Data': 'Comp.pagto.', 'valor_arrecadado': 'Result. pago'})
# df_pred_rev['col'] = df_pred_rev['nome_rubrica'] + '-' + df_pred_rev['vinculo'].astype(str)
# df_pred_rev['Comp.pagto.'] = df_pred_rev['Comp.pagto.'].dt.to_period('M')
# df_pred_rev = df_pred_rev.groupby(['Comp.pagto.', 'vinculo', 'nome_rubrica'], as_index=False).sum()
# df_pred_rev = df_pred_rev.pivot(index='Comp.pagto.', columns=['vinculo', 'nome_rubrica'], values='valor_arrecadado')
# df_pred_rev = df_pred_rev.iloc[:-1]
# df_pred_rev.reset_index(inplace=True)
# df_pred_rev

In [ ]:
# df_pred_exp = pd.concat([df_7001, df_7002, df_7003], ignore_index=True)
# df_pred_exp['col'] = df_pred_exp['Unid. Orçam.'].astype(str) + '-' + df_pred_exp['Proj/Ativ'].astype(str) + '-' + df_pred_exp['Elemento'].astype(str) + '-' + df_pred_exp['Vinc. Orçam.'].astype(str)

# df_pred_rev = df_rev.copy().rename(columns={'Data': 'Comp.pagto.', 'valor_arrecadado': 'Result. pago'})
# df_pred_rev['col'] = df_pred_rev['nome_rubrica'] + '-' + df_pred_rev['vinculo'].astype(str)



# df_pred = pd.concat([df_pred_exp, df_pred_rev], ignore_index=True)
# df_pred

In [7]:
# preparing to predict
df_pred_exp = pd.concat([df_7001, df_7002, df_7003], ignore_index=True)
df_pred_exp['col'] = df_pred_exp['Unid. Orçam.'].astype(str) + '-' + df_pred_exp['Proj/Ativ'].astype(str) + '-' + df_pred_exp['Elemento'].astype(str) + '-' + df_pred_exp['Vinc. Orçam.'].astype(str)

df_pred_rev = df_rev.copy().rename(columns={'Data': 'Comp.pagto.', 'valor_arrecadado': 'Result. pago'})
df_pred_rev['col'] = df_pred_rev['nome_rubrica'] + '-' + df_pred_rev['vinculo'].astype(str)

df_pred = pd.concat([df_pred_exp, df_pred_rev], ignore_index=True)
df_pred['Comp.pagto.'] = df_pred['Comp.pagto.'].dt.to_period('M')
df_pred = df_pred[['Comp.pagto.', 'col', 'Result. pago']].groupby(['Comp.pagto.', 'col'], as_index=False).sum()
df_pred = df_pred.pivot(index='Comp.pagto.', columns='col', values='Result. pago')
df_pred = df_pred.iloc[:-1]
df_pred.reset_index(inplace=True)
df_pred['total_7001'] = df_pred.loc[:, df_pred.columns.str.startswith('7001')].sum(axis=1)
df_pred['total_7002'] = df_pred.loc[:, df_pred.columns.str.startswith('7002')].sum(axis=1)
df_pred['total_7003'] = df_pred.loc[:, df_pred.columns.str.startswith('7003')].sum(axis=1)
df_pred['total_7002_7003'] = df_pred.loc[:, df_pred.columns[df_pred.columns.str.match(r'^(7002|7003)')]].sum(axis=1)
df_pred.loc[df_pred['Comp.pagto.'].dt.year < 2012, 'total_7001'] = np.nan
df_pred['T'] = np.arange(1, len(df_pred)+1)
df_pred = fill_zero(df_pred)

In [195]:
df_pred

col,Comp.pagto.,7001-2529-332039-6069,7001-2529-339014-6069,7001-2529-339030-6069,7001-2529-339033-6069,7001-2529-339035-6069,7001-2529-339037-6069,7001-2529-339039-6069,7001-2529-339040-6069,7001-2529-339092-6069,...,Restituições de Sobra de Adiantamento de Numerário - RPPS-6069,Serv. de Cópias Xerográf. e/ou Cópias Heliográf. - Taxa de Adm. RPPS-6069,Sobra de Adiantamento de Numerário - Taxa de Administração do RPPS-6069,Taxa de Administração - INTRA ORCAMENTÄRIA-6069,Títulos de Responsabilidade do Governo Federal-6050,total_7001,total_7002,total_7003,total_7002_7003,T
0,2011-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.848494e+07,46401.77,3.853134e+07,1
1,2011-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.843061e+07,43398.59,3.847400e+07,2
2,2011-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.883641e+07,43031.63,3.887944e+07,3
3,2011-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.883930e+07,48741.53,3.888805e+07,4
4,2011-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.004686e+07,46555.09,4.009342e+07,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2024-11,0.0,1870.05,531.40,0.00,0.0,12395.82,41577.79,25194.17,0.0,...,0.0,0.0,177.00,20900.0,0.0,3238187.92,1.395900e+08,12826019.49,1.524160e+08,167
167,2024-12,0.0,6759.75,948.00,116.55,14300.0,42340.58,50248.00,25083.27,0.0,...,0.0,0.0,852.60,20900.0,0.0,4122819.14,1.990043e+08,19282831.81,2.182871e+08,168
168,2025-01,0.0,0.00,0.00,40.95,14300.0,12995.43,45476.14,25083.27,0.0,...,0.0,0.0,0.00,0.0,0.0,4185598.14,1.331260e+08,12943186.89,1.460692e+08,169
169,2025-02,0.0,1573.64,1698.00,15737.01,14300.0,23759.29,41768.34,19573.99,452.5,...,0.0,0.0,0.00,0.0,0.0,2945809.16,1.337728e+08,12955992.48,1.467288e+08,170


In [8]:
# prediction interval
start = df_pred['Comp.pagto.'].max()
current_year = start.year if start.month != 12 else start.year + 1
target = pd.Period(f'{current_year + 4}-12', freq='M')
month_diff = (target - start).n
last_T = df_pred['T'].max()
X_prev = pd.DataFrame({'T': [last_T + val for val in range(1, month_diff + 1)]})
years = list(range(current_year, current_year+5))
months = [start.to_timestamp() + pd.DateOffset(months=i) for i in range(1, month_diff + 1)]

In [9]:
def run_models(df: pd.DataFrame, years: list[int], X_prev: pd.DataFrame, start: pd.Period, months: list[pd.Timestamp], month_diff: int, is_ma: bool = False) -> pd.DataFrame:
  df_models = pd.DataFrame({
      'Allocation': pd.Series(dtype='object'),
      'Model': pd.Series(dtype='object'),
      'R²': pd.Series(dtype='float'),
      'RMSE': pd.Series(dtype='float'),
      'MAE': pd.Series(dtype='float'),
      'Forecast': pd.Series(dtype='object'),
      **{year: pd.Series(dtype='float') for year in years}
  })

  def run_linear_models(X: np.ndarray | pd.DataFrame, y: np.ndarray | pd.Series, df_aux: pd.DataFrame, model: str, col: str) -> pd.DataFrame:
    # df_xy = pd.DataFrame({'X': X.flatten(), 'y': y.values})

    # df_train, df_test = time_based_train_test_split(df_xy)
    # if df_train is None:
    #     return df_aux

    lr_model = LinearRegression()
    # lr_model.fit(df_train[['X']], df_train['y'])

    # # Evaluate
    # if df_test is not None:
    #     y_pred_test = lr_model.predict(df_test[['X']])
    #     r2 = r2_score(df_test['y'], y_pred_test)
    #     RMSE = np.sqrt(mean_squared_error(df_test['y'], y_pred_test))
    #     MAE = mean_absolute_error(df_test['y'], y_pred_test)
    # else:
    #     y_pred_train = lr_model.predict(df_train[['X']])
    #     r2 = r2_score(df_train['y'], y_pred_train)
    #     RMSE = np.sqrt(mean_squared_error(df_train['y'], y_pred_train))
    #     MAE = mean_absolute_error(df_train['y'], y_pred_train)

    lr_model.fit(X, y)
    forecast = lr_model.predict(X)
    r2 = r2_score(y, forecast)
    RMSE = np.sqrt(mean_squared_error(y, forecast))
    MAE = mean_absolute_error(y, forecast)

    if model == 'lin-lin':
      y_pred = lr_model.predict(X_prev.values)
    elif model == 'log-log':
      y_pred = np.exp(lr_model.predict(np.log(X_prev).values))
    elif model == 'lin-log':
      y_pred = np.exp(lr_model.predict(X_prev.values))
    elif model == 'log-lin':
      y_pred = lr_model.predict(np.log(X_prev).values)
    else:
      raise ValueError(f"Unknown model type: {model}")

    forecast_df = pd.DataFrame({'date': months, 'y_pred': y_pred})
    forecast_df['year'] = forecast_df['date'].dt.year
    forecast_df['month'] = forecast_df['date'].dt.month

    if is_ma:
      december_forecasts = forecast_df[forecast_df['month'] == 12]
      forecast_years = {year: val * 12 for year, val in zip(december_forecasts['year'], december_forecasts['y_pred'])}
    else:
      forecast_years = forecast_df.groupby('year')['y_pred'].sum().to_dict()
      if start.year in forecast_years:
        forecast_years[start.year] += df.loc[df['Comp.pagto.'].dt.year == start.year, col].sum()

    if any(v < 0 for v in forecast_years.values()):
      return df_aux

    model_dict = {'Allocation': col, 'Model': model, 'R²': r2, 'RMSE': RMSE, 'MAE': MAE, 'Forecast': [forecast_df]}
    model_dict.update(forecast_years)
    df_aux = pd.concat([df_aux, pd.DataFrame(model_dict, index=[0])], ignore_index=True)
    return df_aux


  def run_prophet_model(df_aux: pd.DataFrame, col:str) -> pd.DataFrame:
    prophet_df = df[['Comp.pagto.', col]].copy().rename(columns={
        'Comp.pagto.': 'ds',
        col: 'y'
    }).dropna()

    # adjustment for mass segregation
    if col in mass_segregation_cols:
        prophet_df.loc[prophet_df['ds'] < '2022-05', 'y'] = np.nan
        prophet_df.dropna(inplace=True)

    # prophet_df = df[['Comp.pagto.', col]].rename(columns={
    #     'Comp.pagto.': 'ds',
    #     col: 'y'
    # }).dropna().reset_index(drop=True)

    if len(prophet_df) < 3:
            return df_aux

    prophet_df['ds'] = prophet_df['ds'].dt.to_timestamp()
    # prophet_df['floor'] = 0
    # prophet_df['cap'] = prophet_df['y'].max() * 1.1

    # df_train, df_test = time_based_train_test_split(prophet_df)
    # if df_train is None:
    #     return df_aux

    # len_test = len(df_test) if df_test is not None else 0

    model = build_prophet_model()

    # model = Prophet(yearly_seasonality=True)
    model.fit(prophet_df)
    # model.fit(df_train)

    # future = model.make_future_dataframe(periods=month_diff+len_test, freq='MS')
    future = model.make_future_dataframe(periods=month_diff, freq='MS')
    # future['floor'] = 0
    # future['cap'] = prophet_df['cap'].iloc[0]
    forecast = model.predict(future)

    # model.plot(forecast)

    # display(forecast)

    forecast_train = forecast[forecast['ds'].isin(prophet_df['ds'])]
    r2 = r2_score(prophet_df['y'], forecast_train['yhat'])
    RMSE = np.sqrt(mean_squared_error(prophet_df['y'], forecast_train['yhat']))
    MAE = mean_absolute_error(prophet_df['y'], forecast_train['yhat'])

    # if df_test is not None:
    #     forecast_test = forecast[['ds', 'yhat']].merge(df_test, on='ds', how='inner')
    #     r2 = r2_score(forecast_test['y'], forecast_test['yhat'])
    #     RMSE = np.sqrt(mean_squared_error(forecast_test['y'], forecast_test['yhat']))
    #     MAE = mean_absolute_error(forecast_test['y'], forecast_test['yhat'])
    # else:
    #     forecast_train = forecast[forecast['ds'].isin(prophet_df['ds'])]
    #     r2 = r2_score(prophet_df['y'], forecast_train['yhat'])
    #     RMSE = np.sqrt(mean_squared_error(prophet_df['y'], forecast_train['yhat']))
    #     MAE = mean_absolute_error(prophet_df['y'], forecast_train['yhat'])

    forecast_future = forecast[forecast['ds'] > prophet_df['ds'].max()]

    if is_ma:
      december_forecasts = forecast_future[forecast_future['ds'].dt.month == 12]
      forecast_years = {year: val * 12 for year, val in zip(december_forecasts['ds'].dt.year, december_forecasts['yhat'])}
    else:
      forecast_years = forecast_future.groupby(forecast_future['ds'].dt.year)['yhat'].sum().to_dict()
      if start.year in forecast_years:
        partial_actual_sum = prophet_df[prophet_df['ds'].dt.year == start.year]['y'].sum()
        forecast_years[start.year] += partial_actual_sum

    if any(v < 0 for v in forecast_years.values()):
      return df_aux

    model_dict = {'Allocation': col, 'Model': 'prophet', 'R²': r2, 'RMSE': RMSE, 'MAE': MAE, 'Forecast': [forecast]}
    model_dict.update(forecast_years)
    df_aux = pd.concat([df_aux, pd.DataFrame(model_dict, index=[0])], ignore_index=True)
    return df_aux

  for col in df.columns.difference(['T', 'Comp.pagto.']):
    first_valid = df[col].first_valid_index()

    # adjustment for mass segregation
    if col in mass_segregation_cols:
      first_valid = df.loc[df['Comp.pagto.'] > '2022-04', col].first_valid_index()

    if first_valid is not None:
      X = df['T'][first_valid:].values.reshape(-1,1)
      y = df[col][first_valid:]

      if y.eq(0).sum() / len(y) > 0.8:
        continue

      df_aux = pd.DataFrame()

      # lin-lin
      df_aux = run_linear_models(X, y, df_aux, 'lin-lin', col)

      # log-log
      if (y > 0).all():
        df_aux = run_linear_models(np.log(X), np.log(y), df_aux, 'log-log', col)

      # lin-log
      if (y > 0).all():
        df_aux = run_linear_models(X, np.log(y), df_aux, 'lin-log', col)

      # log-lin
      df_aux = run_linear_models(np.log(X), y, df_aux, 'log-lin', col)

      # prophet
      df_aux = run_prophet_model(df_aux, col)

      if df_aux.empty:
        continue

      # select the best model
      df_aux = df_aux.sort_values(by=['Allocation', 'R²', 'RMSE', 'MAE'], ascending=[True, False, True, True])
      df_models = pd.concat([df_models, df_aux], ignore_index=True)

  return df_models

In [10]:
# Run models separately
models_raw = run_models(df_pred, years, X_prev, start, months, month_diff)
models_ma12 = run_models(moving_averages(df_pred, 12), years, X_prev, start, months, month_diff, True)
models_ma36 = run_models(moving_averages(df_pred, 36), years, X_prev, start, months, month_diff, True)

# Combined results
combined = pd.concat([models_raw, models_ma12, models_ma36], keys=['raw', 'ma12', 'ma36'])
combined_normalized = combined.groupby('Allocation').apply(normalizes_calculates_score, include_groups=False)
combined_normalized.reset_index(inplace=True)
combined_normalized.drop(columns=['level_2'], inplace=True)

# chosen models
chosen_models = (
    combined_normalized
    .sort_values('score', ascending=False)
    .groupby('Allocation')
    .first()
    .rename(columns={'level_1': 'Type'})
)

A saída de streaming foi truncada nas últimas 5000 linhas.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgk0xwxub/lu13hcmj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=88594', 'data', 'file=/tmp/tmpgk0xwxub/fbjivpgi.json', 'init=/tmp/tmpgk0xwxub/lu13hcmj.json', 'output', 'file=/tmp/tmpgk0xwxub/prophet_model4yinblbj/prophet_model-20250415191933.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
19:19:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:19:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgk0xwxub/0jxrd38n.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgk0xwxub/mcn4swb2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib

In [11]:
chosen_models.drop('Forecast', axis=1).to_excel('chosen_models.xlsx')

In [ ]:
models_debug = run_models(moving_averages(df_pred[['Comp.pagto.', 'T', '7001-4396-319011-6069', '7002-2740-319001-6049', '7003-2764-319001-6050', 'total_7002_7003']], 12), years, X_prev, start, months, month_diff)

In [ ]:
models_debug

In [185]:
df_pred_rev = df_rev.copy().rename(columns={'Data': 'Comp.pagto.'})
df_pred_rev['Comp.pagto.'] = df_pred_rev['Comp.pagto.'].dt.to_period('M')
df_pred_rev = df_pred_rev.groupby(['Comp.pagto.', 'vinculo', 'nome_rubrica'], as_index=False).sum()
df_pred_rev = df_pred_rev.pivot(index='Comp.pagto.', columns=['vinculo', 'nome_rubrica'], values='valor_arrecadado')
df_pred_rev = df_pred_rev.iloc[:-1]
df_pred_rev.reset_index(inplace=True)
df_pred_rev

vinculo      Comp.pagto.  \
nome_rubrica               
0                2018-01   
1                2018-02   
2                2018-03   
3                2018-04   
4                2018-05   
..                   ...   
82               2024-11   
83               2024-12   
84               2025-01   
85               2025-02   
86               2025-03   

vinculo                                                                        6049  \
nome_rubrica Compensações Financ entre o Regime Geral e os RPPS-Plano em Repartição   
0                                                    1030508.37                       
1                                                    1051297.19                       
2                                                    1015551.50                       
3                                                    1115195.82                       
4                                                    1000762.92                       
..                                                          ...                       
82                                                   1992544.30                       
83                                                   4962798.00                       
84                                                   1889808.50                       
85                                                   3433933.77                       
86                                                   3307031.03                       

vinculo                                                                     \
nome_rubrica Contr. Patronal - Servidor Ativo - Plano em Repartição - CMPA   
0                                                     387003.83              
1                                                     374983.41              
2                                                     375640.76              
3                                                     368521.98              
4                                                     358057.19              
..                                                          ...              
82                                                   1390574.49              
83                                                   1249625.40              
84                                                    339214.53              
85                                                          NaN              
86                                                    397835.37              

vinculo                                                                             \
nome_rubrica Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada   
0                                                    9254835.20                      
1                                                    8520261.65                      
2                                                    8564117.53                      
3                                                    8327698.67                      
4                                                    7939015.15                      
..                                                          ...                      
82                                                   4277517.72                      
83                                                   9110526.79                      
84                                                   4283528.19                      
85                                                   4240959.47                      
86                                                   4234090.27                      

vinculo                                                                       \
nome_rubrica Contr. Patronal - Servidor Ativo - Plano em Repartição - DEMHAB   
0                                                     346096.90                
1                                                     149645.64                
2                                                

In [186]:
len(df_rev)

7876

In [145]:
len(df_rev) - 4

7876

In [175]:
rubricas = df_pred_rev.columns.get_level_values('nome_rubrica').str.lower()
mask = rubricas.str.contains('682')# & ~rubricas.str.contains('outr') & ~rubricas.str.contains('afast')
df_pred_rev.loc[:, df_pred_rev.columns[mask]]

vinculo                                                                        6050  \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - Centralizada   
Data                                                                                  
2018-01                                                     NaN                       
2018-02                                                     NaN                       
2018-03                                                     NaN                       
2018-04                                                     NaN                       
2018-05                                                     NaN                       
...                                                         ...                       
2024-11                                                     NaN                       
2024-12                                              1017166.12                       
2025-01                                               518527.81                       
2025-02                                               519359.37                       
2025-03                                               528764.29                       

vinculo                                                                        \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - DEMHAB   
Data                                                                            
2018-01                                                     NaN                 
2018-02                                                     NaN                 
2018-03                                                     NaN                 
2018-04                                                     NaN                 
2018-05                                                     NaN                 
...                                                         ...                 
2024-11                                                  449.98                 
2024-12                                                     NaN                 
2025-01                                                  460.79                 
2025-02                                                  461.53                 
2025-03                                                  469.89                 

vinculo                                                                      \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - DMLU   
Data                                                                          
2018-01                                                     NaN               
2018-02                                                     NaN               
2018-03                                                     NaN               
2018-04                                                     NaN               
2018-05                                                     NaN               
...                                                         ...               
2024-11                                                  294.68               
2024-12                                                  751.19               
2025-01                                                  301.76               
2025-02                                                  302.24               
2025-03                                                  307.71               

vinculo                                                                      \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - FASC   
Data                                                                          
2018-01                                                     NaN               
2018-02                                                     NaN               
2018-03                                                     NaN               
2018-04                                                     NaN  

In [155]:
df_pred_rev.to_excel('df_rev.xlsx')

In [ ]:
df_rev[df_rev[['vinculo', 'nome_rubrica']].duplicated(keep='first')]

,Data,vinculo,origem,tipo,nome_rubrica,valor_arrecadado
2961,2017-01-01,6049,Contribuições,Contr.Patronal - Servidor Civil Ativo - Multas...,Contr. Patronal - Servidor Ativo - Plano em Re...,26674.23
2962,2017-02-01,6050,Receita Patrimonial,Rem. dos Rec. do Reg. Próprio de Prev. Social ...,Fundos de Investimentos em Renda Fixa,14529306.33
2963,2017-02-01,6050,Contribuições Intra-Orçamentárias,Contr.Patronal - Servidor Civil Inativo - Prin...,Contr.Patr. - Servidor Civil Inativo - Plano e...,25835.61
2964,2017-02-01,6049,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Repar...,7663210.63
2965,2017-02-01,6050,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Capit...,4859801.46
...,...,...,...,...,...,...
13136,2025-04-01,6050,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em C...,264.58
13137,2025-04-01,6049,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,679.17
13138,2025-04-01,6049,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em R...,1186.11
13139,2025-04-01,6050,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,30.25


In [ ]:
df_rev.loc[df_rev[['vinculo', 'nome_rubrica']] == 6050, , 'nome_rubrica']

In [19]:
for i in df_rev[['vinculo', 'origem', 'tipo', 'nome_rubrica']].drop_duplicates().index:
  if i not in df_rev[['vinculo', 'nome_rubrica']].drop_duplicates().index:
    print(df_rev.iloc[i]['nome_rubrica'], i)

In [12]:
df_rev

,Data,vinculo,origem,tipo,nome_rubrica,valor_arrecadado
2939,2017-01-01,6049,Contribuições,Contr.Patronal - Servidor Civil Ativo - Multas...,Contr. Patronal - Servidor Ativo - Multas e Juros,44.53
2940,2017-01-01,6050,Contribuições Intra-Orçamentárias,Contr.Patronal - Parcelamentos - Servidor Civi...,Parcelamento de Débitos - Patronal,795293.70
2941,2017-01-01,6049,Contribuições,Contr.do Servidor Civil Ativo - Multas e Juros,Contr.do Servidor Civil Ativo - Multas e Juros,37.59
2942,2017-01-01,6050,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Ca...,1756721.26
2943,2017-01-01,6049,Receita de Serviços,Serv.Adm.e Com.Gerais Prest. Entidades e Órgão...,Serv. de Cópias Xerográf. e/ou Cópias Heliográ...,217.39
...,...,...,...,...,...,...
13136,2025-04-01,6050,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em C...,264.58
13137,2025-04-01,6049,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,679.17
13138,2025-04-01,6049,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em R...,1186.11
13139,2025-04-01,6050,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,30.25


In [13]:
df_rev

,Data,vinculo,origem,tipo,nome_rubrica,valor_arrecadado
2939,2017-01-01,6049,Contribuições,Contr.Patronal - Servidor Civil Ativo - Multas...,Contr. Patronal - Servidor Ativo - Multas e Juros,44.53
2940,2017-01-01,6050,Contribuições Intra-Orçamentárias,Contr.Patronal - Parcelamentos - Servidor Civi...,Parcelamento de Débitos - Patronal,795293.70
2941,2017-01-01,6049,Contribuições,Contr.do Servidor Civil Ativo - Multas e Juros,Contr.do Servidor Civil Ativo - Multas e Juros,37.59
2942,2017-01-01,6050,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Ca...,1756721.26
2943,2017-01-01,6049,Receita de Serviços,Serv.Adm.e Com.Gerais Prest. Entidades e Órgão...,Serv. de Cópias Xerográf. e/ou Cópias Heliográ...,217.39
...,...,...,...,...,...,...
13136,2025-04-01,6050,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em C...,264.58
13137,2025-04-01,6049,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,679.17
13138,2025-04-01,6049,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em R...,1186.11
13139,2025-04-01,6050,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,30.25


In [20]:
df_rev[['vinculo', 'origem', 'tipo', 'nome_rubrica']].drop_duplicates()

,vinculo,origem,tipo,nome_rubrica
3264,6049,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Re...
3265,6050,Contribuições Intra-Orçamentárias,Contr. Patronal - Servidor Civil Ativo - Princ...,Contr. Patronal - Servidor Ativo - Plano em Ca...
3266,6050,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Capit...
3267,6050,Contribuições,Contr.do Servidor Civil - Pensionistas - Princ...,Contr.do Servidor Civil - Pensionistas - Plano...
3268,6050,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Capit...
...,...,...,...,...
12664,6049,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg.Próp.Prev....,Comp.Finan.RPPS/RPPS-Estados-Plano em Repartiç...
12921,6049,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Afastado - Pla...
12963,6050,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Afastado - Pla...
12998,6050,Contribuições Intra-Orçamentárias,Contr.Patronal - Parcelamentos - Servidor Civi...,Parc.de Débitos-Patronal-LC nº 637/2010-DEMHAB...


In [21]:
df_rev[['vinculo', 'nome_rubrica']].drop_duplicates()

,vinculo,nome_rubrica
3264,6049,Contr. Patronal - Servidor Ativo - Plano em Re...
3265,6050,Contr. Patronal - Servidor Ativo - Plano em Ca...
3266,6050,Contr.do Servidor Civil Ativo - Plano em Capit...
3267,6050,Contr.do Servidor Civil - Pensionistas - Plano...
3268,6050,Contr.do Servidor Civil Ativo - Plano em Capit...
...,...,...
12664,6049,Comp.Finan.RPPS/RPPS-Estados-Plano em Repartiç...
12921,6049,Contr.do Servidor Civil Ativo - Afastado - Pla...
12963,6050,Contr.do Servidor Civil Ativo - Afastado - Pla...
12998,6050,Parc.de Débitos-Patronal-LC nº 637/2010-DEMHAB...


In [ ]:
df_rev.groupby(['Data', 'vinculo', 'origem', 'tipo', 'nome_rubrica'], as_index=False).sum()

,Data,vinculo,origem,tipo,nome_rubrica,valor_arrecadado
0,2017-01,6049,Contribuições,Contr.Patronal - Servidor Civil Ativo - Multas...,Contr. Patronal - Servidor Ativo - Multas e Juros,44.53
1,2017-01,6049,Contribuições,Contr.Patronal - Servidor Civil Ativo - Multas...,Contr. Patronal - Servidor Ativo - Plano em Re...,26674.23
2,2017-01,6049,Contribuições,Contr.do Servidor Civil - Pensionistas - Princ...,Contr.do Servidor Civil - Pensionistas - Plano...,393746.74
3,2017-01,6049,Contribuições,Contr.do Servidor Civil Ativo - Multas e Juros,Contr.do Servidor Civil Ativo - Multas e Juros,37.59
4,2017-01,6049,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Plano em Repar...,6898791.28
...,...,...,...,...,...,...
10036,2025-04,6050,Contribuições,Contr.do Servidor Civil Ativo - Principal,Contr.do Servidor Civil Ativo - Cedido - Plano...,690.44
10037,2025-04,6050,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Estados - Plano em Ca...,30162.44
10038,2025-04,6050,Outras Receitas Correntes,Comp. Finan. entre o Reg. Geral/Reg. Próp. Pre...,Comp. Finan. RPPS/RPPS - Municípios - Plano em...,30.25
10039,2025-04,6050,Outras Receitas Correntes,Outras Restituições - Principal,Restituições de Servidores - RPPS - Plano em C...,264.58


In [ ]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10074 entries, 2939 to 13140
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype    
---  ------            --------------  -----    
 0   Data              10074 non-null  period[M]
 1   vinculo           10074 non-null  int64    
 2   origem            10074 non-null  object   
 3   tipo              10074 non-null  object   
 4   nome_rubrica      10074 non-null  object   
 5   valor_arrecadado  10074 non-null  float64  
dtypes: float64(1), int64(1), object(3), period[M](1)
memory usage: 550.9+ KB


In [ ]:
len(df_rev.loc[(df_rev['vinculo'].isin([6050, 6069, 6049, 400])) & (df_rev['Data'].dt.year > 2016), 'nome_rubrica'].unique())

206

In [ ]:
df_rev[['vinculo', 'origem', 'tipo', 'nome_rubrica']].drop_duplicates().index

Index([ 2939,  2940,  2941,  2942,  2943,  2944,  2946,  2947,  2948,  2950,
       ...
       12216, 12242, 12412, 12619, 12645, 12664, 12921, 12963, 12998, 13017],
      dtype='int64', length=246)

In [ ]:
df_rev[['vinculo', 'nome_rubrica']].drop_duplicates()

,vinculo,nome_rubrica
2939,6049,Contr. Patronal - Servidor Ativo - Multas e Juros
2940,6050,Parcelamento de Débitos - Patronal
2941,6049,Contr.do Servidor Civil Ativo - Multas e Juros
2942,6050,Contr. Patronal - Servidor Ativo - Plano em Ca...
2943,6049,Serv. de Cópias Xerográf. e/ou Cópias Heliográ...
...,...,...
12664,6049,Comp.Finan.RPPS/RPPS-Estados-Plano em Repartiç...
12921,6049,Contr.do Servidor Civil Ativo - Afastado - Pla...
12963,6050,Contr.do Servidor Civil Ativo - Afastado - Pla...
12998,6050,Parc.de Débitos-Patronal-LC nº 637/2010-DEMHAB...


In [ ]:
# Get the set of unique vinculo + nome_rubrica pairs
simpler_set = df_rev[['vinculo', 'nome_rubrica']].drop_duplicates()

# Get the full 4-column unique rows
full_set = df_rev[['vinculo', 'origem', 'tipo', 'nome_rubrica']].drop_duplicates()

# Find which rows in the full_set are not in the simpler_set
mask = ~full_set[['vinculo', 'nome_rubrica']].apply(tuple, axis=1).isin(
    simpler_set.apply(tuple, axis=1)
)

# Filter the 3 rows
difference_rows = full_set[mask]



In [ ]:
display(mask)

,0
2939,False
2940,False
2941,False
2942,False
2943,False
...,...
12664,False
12921,False
12963,False
12998,False


In [ ]:
len(df_rev['nome_rubrica'].unique())

216

In [ ]:
sorted(df_rev['nome_rubrica'].unique())

['Aluguéis Diversos - Taxa de Administração - INTRA ORCAMENTÄRIA',
 'Aportes Periódicos para Amortização de Déficit Atuarial do RPPS',
 'Cessão do Direito de Operac. da FOPAG',
 'Comp. Finan. RPPS/RPPS - Estados - Plano em Capitalização',
 'Comp. Finan. RPPS/RPPS - Estados - Plano em Repartição',
 'Comp. Finan. RPPS/RPPS - Municípios - Plano em Capitalização',
 'Comp. Finan. RPPS/RPPS - Municípios - Plano em Repartição',
 'Comp. Finan. RPPS/RPPS - União - Plano em Capitalização',
 'Comp. Financ. entre o Regime Geral e os RPPS - Plano em Capitalização',
 'Comp.Finan. Reg. Geral/RPPS - Plano em Capitalização-Multas e Juros',
 'Comp.Finan. Reg.Geral/RPPS - Plano Repartição - Multas e Juros',
 'Comp.Finan.RPPS/RPPS - Municípios - Plano em Repartição-Multas e Juros',
 'Comp.Finan.RPPS/RPPS-Estados-Plano em Capitalização-Multas e Juros',
 'Comp.Finan.RPPS/RPPS-Estados-Plano em Repartição-Multas e Juros',
 'Comp.Finan.RPPS/RPPS-Municípios-Plano em Capitalização-Multas e Juros',
 'Compensações